In [1]:
#Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
# %matplotlib inline
import plotly.express as px
import plotly.io as pio

#Import COSMIC and other things
import cosmic
from cosmic.sample.initialbinarytable import InitialBinaryTable
from cosmic.evolve import Evolve
from cosmic.sample.sampler import independent

In [2]:
#Defining function for rv_variability and filtering
def rv_variable(m1, m2, a, period, ecc, sin_i):
    """
    Function to calculate readial velocity variability
    
    m1: Mass 1
    m2: Mass 2
    period: Period
    ecc: Eccentricity
    a: amplitude
    """
    var = (2*np.pi*a*m2*sin_i)/(period*(m1+m2)*(1-ecc**2)**(1/2))
    return var

def filter_func(m1 , m2, period , ecc, a):
    """
    Function that filters out unwanted values for each parameter 
    Such as 0,-1,inf,nan
    """
    p = period[period != 0]
    p = p[p != -1]
    p = p[p != np.inf]
    
    semi = a[a != 0]
    semi = semi[semi != -1]
    
    e = ecc[ecc != -1]
    
    #Start the filtering!
    #Can check the del_arr as it goes by uncommenting the print lines
    
    #period indecies
    x = period.index[period == 0]
    y = period.index[period == -1]
    z = period.index[period == np.inf]

    #Update del_arr
    del_arr = x
    del_arr = del_arr.append(y)
    del_arr = del_arr.append(z)
    #print(del_arr)

    #Semi major indecies
    x_2 = a.index[a == 0]
    y_2 = a.index[a == -1]

    #Update del_arr
    del_arr = del_arr.append(x_2)
    del_arr = del_arr.append(y_2)
    #print(del_arr)

    #Ecc indecies
    x_3 = ecc.index[ecc == -1]

    #Update del_arr
    del_arr = del_arr.append(x_3)
    #print(del_arr)
    
    #Create final array and remove duplicates
    delete_arr = np.unique(del_arr)

    ecc_f = []
    for i in range(len(ecc)):
        if ecc.index[i] not in delete_arr:
            ecc_f.append(ecc[ecc.index[i]])
    period_f = []
    for i in range(len(period)):
        if period.index[i] not in delete_arr:
            period_f.append(period[period.index[i]])
    a_f = []
    for i in range(len(a)):
        if a.index[i] not in delete_arr:
            a_f.append(a[a.index[i]])
    
    #Update the masses
    m1 = []
    for i in range(len(mass1)):
        if mass1.index[i] not in delete_arr:
            m1.append(mass1[mass1.index[i]])
    m2 = []
    for i in range(len(mass2)):
        if mass2.index[i] not in delete_arr:
            m2.append(mass2[mass2.index[i]])
            
    return m1, m2, period_f, ecc_f, a_f

In [4]:
size = 100000
#Set Initial binary parameters

#Setting what evolution types are allowed
final_kstar1 = [10,11,12]
final_kstar2 = [10,11,12]

#Set the initial binary population parameters
InitialBinaries, mass_singles, mass_binaries, n_singles, n_binaries = \
     InitialBinaryTable.sampler('independent', final_kstar1, final_kstar2, binfrac_model=1.,
                                primary_model='kroupa01', ecc_model='sana12', porb_model='log_uniform',
                                qmin=-1, SF_start=13700.0, SF_duration=0.0, met=0.02, size=size)

#Can print initial binaries here to check
#print(InitialBinaries)

print('####################### Initial Binaries Set ##########################')

#Set the BSEDict
BSEDict = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': 1.0, 'pts1': 0.001, 'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000, 'bwind': 0.0, 'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5, 'grflag' : 1, 'remnantflag': 4, 'ceflag': 0, 'eddfac': 1.0, 'ifflag': 0, 'bconst': 3000, 'sigma': 265.0, 'gamma': -2.0, 'pisn': 45.0, 'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]], 'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90, 'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0], 'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 0, 'ecsn' : 2.25, 'ecsn_mlow' : 1.6, 'aic' : 1, 'ussn' : 0, 'sigmadiv' :-20.0, 'qcflag' : 1, 'eddlimflag' : 0, 'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0], 'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1, 'ST_cr' : 1, 'ST_tide' : 1, 'bdecayfac' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 0, 'zsun' : 0.014, 'bhms_coll_flag' : 0, 'don_lim' : -1, 'acc_lim' : -1, 'dtp' : 13700.0}

#Evolve the system
bpp, bcm, initC, kick_info  = Evolve.evolve(initialbinarytable=InitialBinaries, BSEDict=BSEDict)

print(bcm.iloc[:10])

print('###################### System Evolved ################################')


####################### Initial Binaries Set ##########################


/Users/noahbungart/opt/anaconda3/lib/python3.8/site-packages/cosmic/evolve.py:258: UserWarning: The value for dtp in initial binary table is being overwritten by the value of dtp from either the params file or the BSEDict.
  warnings.warn("The value for {0} in initial binary table is being "


     tphys  kstar_1    mass0_1     mass_1         lum_1     rad_1  \
0      0.0      1.0   1.208488   1.208488  1.567844e+00  1.145982   
0  13700.0     10.0   0.449547   0.449547  1.627586e-04  0.015051   
1      0.0      1.0   1.184163   1.184163  1.423406e+00  1.110837   
1  13700.0     11.0   0.551909   0.551909  4.387021e-05  0.013449   
2      0.0      1.0   1.846615   1.846615  1.059198e+01  1.624182   
2  13700.0     11.0   0.677604   0.677604  5.588559e-06  0.011773   
3      0.0      1.0  13.687247  13.687247  1.514354e+04  5.031111   
3  13700.0     13.0  16.030386   1.879628  1.629947e-10  0.000014   
4      0.0      0.0   0.719722   0.719722  1.309322e-01  0.669691   
4  13700.0      0.0   0.719722   0.719722  1.702767e-01  0.711524   

         teff_1   massc_1    radc_1        menv_1  ...          porb  \
0   6060.470738  0.000000  0.000000  7.023251e-03  ...  1.877815e+01   
0   5338.018652  0.449547  0.015051  1.000000e-10  ...  0.000000e+00   
1   6008.645068  0.00000

In [5]:
#Get all parameters and create sini artificial data
mass1 = bcm.mass_1[bcm.tphys == 13700.0]
#print(mass1)
mass2 = bcm.mass_2[bcm.tphys == 13700.0]
#print(mass2)
period = bcm.porb[bcm.tphys == 13700.0]
#print(period)
ecc = bcm.ecc[bcm.tphys == 13700.0]
#print(ecc)
a = bcm.sep[bcm.tphys == 13700.0]
#print(a)

#Checking to make sure all initial dataframes are the 
#same length
#print(len(mass1))
#print(len(mass2))
#print(len(period))
#print(len(ecc))
#print(len(a))

m1, m2, period_f, ecc_f, a_f = filter_func(mass1, mass2, period, ecc, a)

sini = np.random.uniform(0, 1, len(m1))

print('######################### Filtering Complete #######################')


######################### Filtering Complete #######################


In [21]:
bcm_test = bcm[bcm.tphys == 13700.0]
x= np.array(bcm_test.ecc)
y = np.array(bcm_test.porb)
[i for i in range(len(x)) if (x[i] == 0 and y[i] < 1)]

[12,
 21,
 23,
 31,
 32,
 38,
 42,
 53,
 73,
 119,
 123,
 131,
 133,
 153,
 156,
 158,
 167,
 186,
 197,
 212,
 218,
 219,
 250,
 263,
 305,
 308,
 314,
 323,
 326,
 349,
 358,
 417,
 432,
 436,
 460,
 463,
 469,
 471,
 474,
 487,
 495,
 497,
 501,
 503,
 530,
 575,
 587,
 588,
 593,
 603,
 652,
 665,
 690,
 741,
 742,
 755,
 756,
 757,
 764,
 795,
 805,
 808,
 818,
 853,
 869,
 909,
 924,
 934,
 940,
 971,
 995,
 1029,
 1058,
 1063,
 1116,
 1153,
 1179,
 1192,
 1200,
 1203,
 1230,
 1238,
 1244,
 1274,
 1322,
 1326,
 1331,
 1332,
 1373,
 1384,
 1396,
 1399,
 1411,
 1415,
 1442,
 1461,
 1462,
 1496,
 1519,
 1572,
 1582,
 1586,
 1592,
 1599,
 1623,
 1662,
 1664,
 1676,
 1682,
 1695,
 1697,
 1701,
 1713,
 1725,
 1730,
 1738,
 1743,
 1751,
 1760,
 1769,
 1771,
 1782,
 1788,
 1819,
 1823,
 1856,
 1883,
 1950,
 1951,
 2009,
 2060,
 2061,
 2095,
 2096,
 2108,
 2126,
 2131,
 2134,
 2137,
 2206,
 2208,
 2224,
 2231,
 2272,
 2274,
 2304,
 2331,
 2337,
 2339,
 2347,
 2368,
 2375,
 2383,
 2384,
 2

In [36]:
print("Mass1:\n",bcm.mass_1[12],"\n",
      "",
      "Mass2:\n",bcm.mass_2[12],"\n",
      "",
      "Period:\n",bcm.porb[12],"\n",
      "",
      "Separation:\n",bcm.sep[12],"\n",
      "",
      "Eccentricity:\n",bcm.ecc[12])

Mass1:
 12    1.47930
12    0.51353
Name: mass_1, dtype: float64 
  Mass2:
 12    1.395714
12    0.286518
Name: mass_2, dtype: float64 
  Period:
 12    1334.293295
12       0.122481
Name: porb, dtype: float64 
  Separation:
 12    724.991227
12      0.963168
Name: sep, dtype: float64 
  Eccentricity:
 12    0.159472
12    0.000000
Name: ecc, dtype: float64


In [41]:
sini = np.random.uniform(0, 1, len(bcm_test))
m1 = bcm.mass_1[bcm.tphys == 13700.0][12]
m2 = bcm.mass_2[bcm.tphys == 13700.0][12]
a = bcm.sep[bcm.tphys == 13700.0][12]
p = bcm.porb[bcm.tphys == 13700.0][12]
ecc = bcm.ecc[bcm.tphys == 13700.0][12]

In [44]:
rv = rv_variable(m1, m2, a, p, ecc, sini[12])
print(float(rv))

2.6366802008668078


In [46]:
print(type(rv))

<class 'numpy.float64'>


In [47]:
conversion = 696340 / 86400
print(rv * conversion)

21.25029966518047
